# Phase 1 - Data Engineering : Collecte de Tweets Tesla

Ce notebook démontre la collecte de tweets sur Tesla en utilisant l'API Twitter v2 avec Tweepy.

## Objectifs :
- Se connecter à l'API Twitter
- Collecter 500 tweets récents sur Tesla
- Sauvegarder les données brutes en CSV
- Visualiser les données collectées



In [ ]:
# Importation des bibliothèques nécessaires
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Ajouter le répertoire parent au path pour importer les modules
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

from src.collect_tesla_tweets import TeslaTweetCollector

print("✅ Bibliothèques importées avec succès")



## Étape 1 : Initialisation du collecteur

Le collecteur utilise le Bearer Token pour s'authentifier à l'API Twitter v2.
Les credentials doivent être définis dans un fichier `.env` à la racine du projet.


In [ ]:
# Initialiser le collecteur
# Le token sera chargé depuis le fichier .env
collector = TeslaTweetCollector()

# Tester la connexion à l'API
if collector.test_connection():
    print("✅ Connexion réussie !")
else:
    print("❌ Échec de la connexion. Vérifiez vos credentials.")



## Étape 2 : Collecte des tweets

Nous allons collecter 500 tweets récents sur Tesla. La requête inclut :
- Mots-clés : Tesla, TSLA, @Tesla, "Elon Musk"
- Exclusion des retweets
- Langue : anglais

**Note** : L'API Twitter Essential (gratuite) a des limites de rate limit qui sont gérées automatiquement par Tweepy avec `wait_on_rate_limit=True`.


In [ ]:
# Collecter 500 tweets
# Cela peut prendre quelques minutes selon les rate limits
df_tweets = collector.collect_tweets(max_tweets=500, output_file="../data/tesla_tweets_raw.csv")

# Afficher un aperçu
print(f"\n📊 {len(df_tweets)} tweets collectés")
df_tweets.head()



## Étape 3 : Analyse exploratoire des données brutes

Analysons les données collectées pour comprendre leur structure et qualité.


In [ ]:
# Informations sur le DataFrame
print("📋 Informations du DataFrame :")
print(f"   Nombre de lignes : {len(df_tweets)}")
print(f"   Nombre de colonnes : {len(df_tweets.columns)}")
print(f"\n📊 Colonnes disponibles :")
print(df_tweets.columns.tolist())

# Statistiques descriptives
print("\n📈 Statistiques descriptives :")
df_tweets.describe()



In [ ]:
# Vérifier les dates (période de collecte)
if 'date' in df_tweets.columns:
    df_tweets['date'] = pd.to_datetime(df_tweets['date'])
    print(f"📅 Période de collecte :")
    print(f"   Date la plus ancienne : {df_tweets['date'].min()}")
    print(f"   Date la plus récente : {df_tweets['date'].max()}")
    print(f"   Période couverte : {(df_tweets['date'].max() - df_tweets['date'].min()).days} jours")



In [ ]:
# Visualisation : Distribution temporelle des tweets
if 'date' in df_tweets.columns:
    plt.figure(figsize=(12, 6))
    df_tweets['date'].hist(bins=30, edgecolor='black')
    plt.title('Distribution temporelle des tweets collectés', fontsize=14, fontweight='bold')
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Nombre de tweets', fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()



In [ ]:
# Visualisation : Top 10 utilisateurs par nombre de tweets
if 'user' in df_tweets.columns:
    top_users = df_tweets['user'].value_counts().head(10)
    
    plt.figure(figsize=(10, 6))
    top_users.plot(kind='barh', color='#E31937')
    plt.title('Top 10 utilisateurs par nombre de tweets', fontsize=14, fontweight='bold')
    plt.xlabel('Nombre de tweets', fontsize=12)
    plt.ylabel('Utilisateur', fontsize=12)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()



## Étape 4 : Exemples de tweets collectés

Examinons quelques exemples de tweets pour vérifier la qualité des données.


In [ ]:
# Afficher quelques exemples de tweets
print("📝 Exemples de tweets collectés :\n")
for idx, row in df_tweets.head(5).iterrows():
    print(f"Tweet #{idx+1}:")
    print(f"  Utilisateur: @{row.get('user', 'unknown')}")
    print(f"  Date: {row.get('date', 'N/A')}")
    print(f"  Likes: {row.get('likes', 0)} | RT: {row.get('retweets', 0)}")
    print(f"  Texte: {row['text'][:150]}...")
    print("-" * 80)



: 

## Conclusion

✅ **Données collectées avec succès !**

Les tweets bruts ont été sauvegardés dans `../data/tesla_tweets_raw.csv`.

**Prochaine étape** : Passer au notebook `2_preprocess_analyze.ipynb` pour nettoyer et analyser ces données.

